In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('autos.csv')

In [3]:
def outlier(value):
    Q1 = np.percentile(df[value], 25)
    Q3 = np.percentile(df[value], 75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR

    df[df[value] > upper] = np.nan
    df[df[value] < lower] = np.nan

In [4]:
for i in ['yearOfRegistration', 'powerPS']:
    outlier(i)

In [5]:
from scipy.stats.mstats import winsorize

# # apply Winsorization with 10% trimming on both ends
df['price'] = np.log(winsorize(df['price'], limits=[0.1, 0.1]) + 1)

In [6]:
df.select_dtypes(exclude=object).fillna(df.select_dtypes(exclude=object).mean(), inplace=True)

In [7]:
df.dropna(inplace=True)

In [8]:
df['notRepairedDamage'] = pd.factorize(df['notRepairedDamage'])[0]
df['seller'] = pd.factorize(df['seller'])[0]
df['offerType'] = pd.factorize(df['offerType'])[0]
df['abtest'] = pd.factorize(df['abtest'])[0]
df['gearbox'] = pd.factorize(df['gearbox'])[0]

In [9]:
df_one_hot = pd.get_dummies(df, prefix='vehicleType', prefix_sep='.', 
                            columns=['vehicleType'])
df_one_hot = pd.get_dummies(df_one_hot, prefix='fuelType', prefix_sep='.', 
                            columns=['fuelType'], drop_first=True)
df_one_hot = pd.get_dummies(df_one_hot, prefix='brand', prefix_sep='.', 
                            columns=['brand'], drop_first=True)
df_one_hot = pd.get_dummies(df_one_hot, prefix='model', prefix_sep='.', 
                            columns=['model'], drop_first=True)

df_one_hot['yearOfRegistration'] = df_one_hot['yearOfRegistration'].astype(int)

df_one_hot['year_range'] = pd.cut(df_one_hot['yearOfRegistration'], bins=range(df_one_hot['yearOfRegistration'].min(), df_one_hot['yearOfRegistration'].max()+6, 5), right=False)
df_one_hot = pd.get_dummies(df_one_hot, prefix='year_range', prefix_sep='.', columns=['year_range'], drop_first=True)

df_one_hot.drop(columns=['index', 'name','dateCrawled', 'dateCreated', 'nrOfPictures', 'lastSeen', 'monthOfRegistration', 'postalCode','yearOfRegistration'], inplace=True)

In [10]:
df_one_hot

,seller,offerType,price,abtest,gearbox,powerPS,kilometer,notRepairedDamage,vehicleType.andere,vehicleType.bus,...,model.yeti,model.ypsilon,model.z_reihe,model.zafira,"year_range.[1991, 1996)","year_range.[1996, 2001)","year_range.[2001, 2006)","year_range.[2006, 2011)","year_range.[2011, 2016)","year_range.[2016, 2021)"
3,0,0,7.313887,0,0,75.0,150000.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,8.188967,0,0,69.0,90000.0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,6.478510,0,0,102.0,150000.0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,7.696667,0,0,109.0,150000.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10,0,0,7.601402,1,0,105.0,150000.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371520,0,0,8.071219,1,0,225.0,150000.0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
371521,0,0,7.048386,1,0,0.0,150000.0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
371524,0,0,7.090077,0,1,101.0,125000.0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
371525,0,0,9.127067,0,0,102.0,150000.0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [11]:
df_one_hot.columns

Index(['seller', 'offerType', 'price', 'abtest', 'gearbox', 'powerPS',
       'kilometer', 'notRepairedDamage', 'vehicleType.andere',
       'vehicleType.bus',
       ...
       'model.yeti', 'model.ypsilon', 'model.z_reihe', 'model.zafira',
       'year_range.[1991, 1996)', 'year_range.[1996, 2001)',
       'year_range.[2001, 2006)', 'year_range.[2006, 2011)',
       'year_range.[2011, 2016)', 'year_range.[2016, 2021)'],
      dtype='object', length=313)

In [12]:
scaler = StandardScaler()
df_one_hot['powerPS'] = scaler.fit_transform(df_one_hot[['powerPS']])
df_one_hot['kilometer'] = scaler.fit_transform(df_one_hot[['kilometer']])

In [13]:
df_one_hot

,seller,offerType,price,abtest,gearbox,powerPS,kilometer,notRepairedDamage,vehicleType.andere,vehicleType.bus,...,model.yeti,model.ypsilon,model.z_reihe,model.zafira,"year_range.[1991, 1996)","year_range.[1996, 2001)","year_range.[2001, 2006)","year_range.[2006, 2011)","year_range.[2011, 2016)","year_range.[2016, 2021)"
3,0,0,7.313887,0,0,-0.353911,0.647340,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,8.188967,0,0,-0.395023,-0.867131,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,0,0,6.478510,0,0,-0.168908,0.647340,1,0,0,...,0,0,0,0,1,0,0,0,0,0
6,0,0,7.696667,0,0,-0.120944,0.647340,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10,0,0,7.601402,1,0,-0.148352,0.647340,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371520,0,0,8.071219,1,0,0.673887,0.647340,1,0,0,...,0,0,0,0,0,0,1,0,0,0
371521,0,0,7.048386,1,0,-0.867810,0.647340,0,0,1,...,0,0,0,1,0,1,0,0,0,0
371524,0,0,7.090077,0,1,-0.175760,0.016310,0,0,0,...,0,0,0,0,0,1,0,0,0,0
371525,0,0,9.127067,0,0,-0.168908,0.647340,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [14]:
df_one_hot.to_csv('cleaned_used_cars_v3.csv', index=False)

In [15]:
df_one_hot.columns

Index(['seller', 'offerType', 'price', 'abtest', 'gearbox', 'powerPS',
       'kilometer', 'notRepairedDamage', 'vehicleType.andere',
       'vehicleType.bus',
       ...
       'model.yeti', 'model.ypsilon', 'model.z_reihe', 'model.zafira',
       'year_range.[1991, 1996)', 'year_range.[1996, 2001)',
       'year_range.[2001, 2006)', 'year_range.[2006, 2011)',
       'year_range.[2011, 2016)', 'year_range.[2016, 2021)'],
      dtype='object', length=313)

In [16]:
temp = abs(df_one_hot.corr()['price']).sort_values(ascending=False)
temp

price                      1.000000
year_range.[1996, 2001)    0.517016
year_range.[2011, 2016)    0.437515
kilometer                  0.419300
year_range.[2006, 2011)    0.417563
                             ...   
model.lancer               0.000579
model.c1                   0.000523
model.musa                 0.000472
model.croma                0.000384
model.i3                   0.000116
Name: price, Length: 313, dtype: float64